In [9]:
import sqlite3
import pandas as pd

def load_nsd_to_dataframe(db_name='nsd.db'):
    """
    Carrega os dados do banco de dados nsd.db em um DataFrame do Pandas.

    Parameters:
    - db_name (str): O nome do banco de dados SQLite.

    Returns:
    pd.DataFrame: DataFrame contendo os dados da tabela nsd.
    """
    # Conecte-se ao banco de dados
    conn = sqlite3.connect(db_name)
    
    # Leia os dados da tabela nsd
    query = "SELECT * FROM nsd"
    df = pd.read_sql_query(query, conn)
    
    # Feche a conexão
    conn.close()
    
    return df

# Use a função para carregar os dados
df_nsd = load_nsd_to_dataframe()
df_nsd.head()


,nsd,company,dri,nsd_type,version,auditor,auditor_rt,protocolo,date,sent_date,reason
0,34,BAESAENERGETICA BARRA GRANDE SA,CARLOS ALBERTO BEZERRA DE MIRANDA,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MARCELO LIMA TONINI,019607FCA00002010010000003487,2010-01-01T00:00:00,2010-03-05T16:22:04,
1,35,PRUMO LOGISTICA SA,OTAVIO DE GARCIA LAZCANO,FORMULARIO CADASTRAL,1,KPMG AUDITORES INDEPENDENTES,MANUEL FERNANDES RODRIGUES DE SOUSA,021482FCA00002010010000003588,2010-01-01T00:00:00,2010-03-05T16:35:55,
2,36,CIA SANEAMENTO BASICO EST SAO PAULO,RUI DE BRITTO ALVARES AFFONSO,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,PAULO CESAR ESTEVAO NETTO,014443FCA00002010010000003680,2010-01-01T00:00:00,2010-03-05T18:41:51,
3,37,EVEN CONSTRUTORA E INCORPORADORA SA,DANY MUSZKAT,FORMULARIO CADASTRAL,1,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES,VALDIR RENATO COSCODAI,020524FCA00002010010000003782,2010-01-01T00:00:00,2010-03-09T17:00:41,
4,39,JBS SA,JEREMIAH ALPHONSUS OCALLAGHAN,FORMULARIO CADASTRAL,1,BDO TREVISAN AUDITORES INDEPENDENTES,ESTEFAN GEORGE HADDAD,020575FCA00002010010000003982,2010-01-01T00:00:00,2010-03-11T20:02:44,


In [10]:
df_nsd['nsd_type'].unique()

array(['FORMULARIO CADASTRAL', 'FORMULARIO DE REFERENCIA'], dtype=object)